# Get cell processing info

In [ ]:
import os, sys, glob, time
import numpy as np
import pandas as pd
#import osgeo # needed only if running from windows
import io
import requests
import ee

from shapely.geometry import Polygon, mapping
import geopandas as gpd
import geemap
import geemap.geemap as geemap
import jupyter_contrib_nbextensions 
import ipyleaflet
import ipywidgets as widgets

In [ ]:
## note: need to authenticate through an account with NICFI and developer access
## can access through cel.emlab@gmail.com
ee.Authenticate() # do NOT select read only scopes
ee.Initialize()

In [ ]:
def GET_GEE(gee_name, start, end, aoi_shape, chip_id):
    gdf = gpd.read_file(aoi_shape)
    gdf_web = gdf.to_crs("EPSG:4326")
    gdf_web = gdf_web[gdf_web["chip_id"] == chip_id]
    gdf_web = gdf_web[gdf_web["chip_id"] == chip_id]
    
    aoi = ee.Geometry.Rectangle([gdf_web.bounds.minx.min(), gdf_web.bounds.miny.min(), gdf_web.bounds.maxx.max(), gdf_web.bounds.maxy.max()])

    collection = ee.ImageCollection(gee_name).filterBounds(aoi).filterDate(start, end)
    clip_col = collection.map(lambda col : col.clip(aoi))

    return clip_col


def getNDVI(image):
    ndvi = image.normalizedDifference(['N','R']).rename("NDVI")
    
    return(ndvi)


## JUST DOWNLOAD CHIPS 
def GET_monthly_TS(chip_shape, chip_id, start_date, end_date, export_composite=False):

    planet_NDVI = GET_GEE(gee_name="projects/planet-nicfi/assets/basemaps/americas", 
                 start=start_date, end=end_date, 
                 aoi_shape=chip_shape, 
                 chip_id=chip_id).map(getNDVI).toBands()

    planet_NDVI=ee.Image(planet_NDVI).multiply(10000).int()

    if not os.path.exists(export_composite):
        os.makedirs(export_composite)
    out_file = os.path.join(export_composite, "PS_monthly_"+str(chip_id)+".tif")
    geemap.ee_export_image(planet_NDVI, out_file, scale=4.7)
    return out_file

In [ ]:
## TODO: Set paramaters here. chip_file location & out_dir (where 'images' folder will be placed).
chip_file="C:/GISprojects/PyCropSeg/Priority9chips.shp"
chips = gpd.read_file(chip_file)

for chipID in chips.chip_id.to_list():
    planet_img = GET_monthly_TS(chip_shape=chip_file, 
                                chip_id=chipID, 
                                start_date="2020-07-01", 
                                end_date="2021-08-01", 
                                export_composite="images/")